In [57]:
import random
from enum import Enum
from functools import lru_cache
from math import ceil
from statistics import mean
from typing import NamedTuple

import numpy as np

from deap import algorithms, base, creator, tools

In [58]:
class Subject:
    def __init__(self, name, numbers):
        self.name = name
        self.numbers = numbers

    def __repr__(self): #represent
        return self.name

In [59]:
class SubjectType(Enum):
    #default
    Lecture = 1 
    Lab = 0
    Test = 0
    Speaking=0

In [60]:
class WeekDay(Enum):
    MON, TUE, WED, THU, FRI, SAT, SUN = 0, 1, 2, 3, 4, 5, 6

In [61]:
C1 = 1  # uniformity สม่ำเสมอ
C2 = 1  # tightness density
C3 = 1  # suitability เหมาะสม
C4 = 0.1  # sleep
C5 = 1  # day_grouping
C6 = 1  # week_grouping

In [62]:
classPerDay = 6
workDay = 5
places = classPerDay*workDay

In [63]:
subjects = [
    Subject("521", {SubjectType.Lecture:4, SubjectType.Test:4, SubjectType.Lab:0, SubjectType.Speaking:0}),
    Subject("523", {SubjectType.Lecture:0, SubjectType.Test:0, SubjectType.Lab:4, SubjectType.Speaking:0}),
    Subject("552", {SubjectType.Lecture:2, SubjectType.Test:0, SubjectType.Lab:0, SubjectType.Speaking:1}),
    Subject("525", {SubjectType.Lecture:2, SubjectType.Test:0, SubjectType.Lab:0, SubjectType.Speaking:0}),
    Subject("776", {SubjectType.Lecture:3, SubjectType.Test:0, SubjectType.Lab:0, SubjectType.Speaking:0})
]

In [64]:
# a suitable time for each subject

suitableTime = {
    WeekDay.MON:{1:["521"], 2:[], 3:[], 4:[], 5:[], 6:[]},
    WeekDay.TUE:{1:[], 2:["521"], 3:[], 4:[], 5:[], 6:[]},
    WeekDay.WED:{1:[], 2:[], 3:[], 4:[], 5:[], 6:[]},
    WeekDay.THU:{1:[], 2:[], 3:[], 4:[], 5:[], 6:[]},
    WeekDay.FRI:{1:[], 2:[], 3:[], 4:[], 5:[], 6:[]},
}

In [65]:
class Class(NamedTuple):
    #tuple of dic
    subject: Subject
    type: SubjectType

In [66]:
@lru_cache #decrease runtime
#fitness function
def calcUniformity(numByDay):
    n = sum(numByDay)
    p = workDay
    meanNum = n/p 

    dev = (sum((c-meanNum)**2 for c in numByDay))**0.5

    return dev

In [67]:
def constructSch(ordering, mapping):
    classesNum = len(mapping)
    subjectsOrder = [mapping[o] if o<classesNum else None for o in ordering]

    #1 d
    return [subjectsOrder[i: i+classPerDay] for i in range(0, len(subjectsOrder), classPerDay)]

In [68]:
def evaluteOrder(ordering, classes, mapping):
    classesNum = len(mapping)
    schedule = constructSch(ordering, mapping)
    numByDay = tuple(sum(s is not None for s in day) for day in schedule)

    uniformity = calcUniformity(numByDay)
    return uniformity,

In [69]:
def main():
        lecture = [Class(sub, SubjectType.Lecture) for sub in subjects
                for _ in range(sub.numbers[SubjectType.Lecture])]
        test = [Class(sub, SubjectType.Test) for sub in subjects
                for _ in range(sub.numbers[SubjectType.Test])]
        lab = [Class(sub, SubjectType.Lab) for sub in subjects
                for _ in range(sub.numbers[SubjectType.Lab])]
        speaking = [Class(sub, SubjectType.Speaking) for sub in subjects
                for _ in range(sub.numbers[SubjectType.Speaking])]

        classes = lecture+test+lab+speaking
        mapping = {i:class_ for i,class_ in enumerate(classes)}
        #print(mapping)

        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        toolbox = base.Toolbox()

        toolbox.register('indices', random.sample, range(places), places)
        toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.indices)
        toolbox.register('population', tools.initRepeat, list, toolbox.individual)

        toolbox.register('evaluate', lambda ordering: evaluteOrder(ordering, classes, mapping))
        toolbox.register('mate', tools.cxOrdered)
        toolbox.register('mutate', tools.mutShuffleIndexes, indpb=0.01)
        toolbox.register('select', tools.selTournament, tournsize=3)

        pop = toolbox.population(n=300)
        hof = tools.HallOfFame(1)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register('avg', np.mean)
        stats.register('std', np.std)
        stats.register('min', np.min)
        stats.register('max', np.max)

        algorithms.eaSimple(
                pop, toolbox,
                ngen=200,
                cxpb=0.5,
                mutpb=0.2,
                stats=stats,
                verbose=True,
                halloffame=hof,
                )

        for day, day_name in zip(constructSch(hof.items[0], mapping), WeekDay):
                print(f'\n{day_name.name}:')
                for i, class_ in enumerate(day):
                        print(f'{i + 1})' + ('' if class_ is None else f' {class_.subject.name} {class_.type.name}'))


if __name__ == '__main__':
        main()

gen	nevals	avg    	std     	min     	max   
0  	300   	2.37529	0.823398	0.894427	4.5607
1  	193   	1.81249	0.654392	0.894427	3.84708
2  	176   	1.5553 	0.653997	0.894427	3.84708
3  	170   	1.37561	0.600857	0.894427	3.84708
4  	185   	1.36316	0.633879	0.894427	4.5607 
5  	177   	1.22729	0.523217	0.894427	3.28634
6  	180   	1.30116	0.577622	0.894427	3.28634
7  	185   	1.25902	0.567046	0.894427	3.57771
8  	188   	1.29403	0.610045	0.894427	3.28634
9  	175   	1.27643	0.608864	0.894427	3.84708
10 	163   	1.19058	0.518842	0.894427	2.96648
11 	182   	1.22886	0.591536	0.894427	3.28634
12 	176   	1.25916	0.589793	0.894427	3.28634
13 	180   	1.25577	0.602524	0.894427	3.84708
14 	191   	1.28713	0.597177	0.894427	3.28634
15 	174   	1.18751	0.49981 	0.894427	3.84708
16 	193   	1.27101	0.598228	0.894427	3.57771
17 	182   	1.24617	0.594747	0.894427	3.28634
18 	197   	1.32612	0.60116 	0.894427	3.28634
19 	167   	1.20709	0.48607 	0.894427	2.96648
20 	202   	1.28385	0.56427 	0.894427	3.84708
21 	169   	1

In [70]:
y = [1, 2, 3, 4, 5, 6]
print(y[:-1], y[1:])
print(zip(y[:-1], y[1:]))
for a,b in zip(y[:-1], y[1:]):
    print(a, b)

[1, 2, 3, 4, 5] [2, 3, 4, 5, 6]
1 2
2 3
3 4
4 5
5 6
